# En proceso

In [ ]:
M, N = 1000, 1000
# Crear un conjunto de datos con distribución log normal
x = np.linspace(0, 1, num=M)
cov = 0.1*np.exp(-0.5*(x[:, None]  - x[:, None].T)**2/0.5**2)
data = np.exp(np.random.multivariate_normal(mean=np.zeros(M), cov=cov, size=N))
# Una función que ajusta un modelo polinomial al logaritmo de los datos
def slow_function(data):
    X = np.vstack([x**k for k in range(4)]).T
    D = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
    mse = np.zeros(shape=(len(data),))
    for i, y in enumerate(data):
        y_log = np.log(y)
        y_mean = np.mean(y_log)
        y_var = np.var(y_log)
        y_norm = (y_log - y_mean)/np.sqrt(y_var)
        theta = np.dot(D, y_norm)
        model = np.dot(X, theta)
        mse[i] = np.mean((y_norm - model)**2)
    return mse

In [ ]:


x = np.linspace(0, 0.1, num=1000)
y = [sum([A[k]*np.cos(2.0*np.pi*freq_*x*(k+1) + phase_[k]) for k in range(4)]) for freq_, phase_ in zip(freq, phase)]
y = np.stack(y)

fig, ax = plt.subplots()
for i, y_ in enumerate(y[0:10]):
    ax.plot(x, y_ + i*3, alpha=0.5)
    
for y_ in y:
    res = scipy.stats.linregress(x, y_)